# Psi v meste (Prešov, Žilina, Martin) 

In [ ]:
from psiapomoc import psiNaUlici, psiMesta, mesta
import panel as pn
import panel.widgets as pnw
pn.extension(template="fast")

### Dáta o psoch sme zobrali z [http://pomosam.sk](http://pomosam.sk)

In [ ]:
mesto_vyber = pnw.RadioButtonGroup(options={"Martin": "martin", "Žilina": "zilina", "Prešov": "presov"}, value="martin", width=500)

ulica_vyber = pnw.Select(options=mesta['martin']['ulice'])

plemeno_vyber = pnw.Select(options=mesta['martin']['plemena'], disabled = True)

zobraz_vyber = pnw.RadioBoxGroup(options={"Ulice": "ulice", "Plemená": "plemena"}, value="ulice")

@pn.depends(mesto_vyber.param.value, watch=True)
def update_data_mesta(mesto_vyber):
    data = mesta[mesto_vyber]
    ulica_vyber.options = data['ulice']
    plemeno_vyber.options = data['plemena']
    if plemeno_vyber.value not in plemeno_vyber.options:
        plemeno_vyber.value = data['plemena'][0]


@pn.depends(zobraz_vyber.param.value, watch=True)
def update_ulice_plemena(zobraz_vyber):
    if zobraz_vyber == "ulice":
        plemeno_vyber.disabled = True
        ulica_vyber.disabled = False
    else:
        ulica_vyber.disabled = True
        plemeno_vyber.disabled = False
        dv = mesta[mesto_vyber.value]
        plemeno_vyber.options = dv['plemena']
        # plemeno_vyber.value = mesta[mesto_vyber.value]['plemena'][0]

In [ ]:
@pn.depends(ulica_vyber, plemeno_vyber, zobraz_vyber,mesto_vyber)
def psi_zobraz(ulica_vyber, plemeno_vyber, zobraz_vyber, mesto_vyber):
    mv = mesto_vyber
    if zobraz_vyber == "ulice":
        res = psiNaUlici(mv, mesta[mv]['psi'], ulica_vyber, mapa=True)
        if type(res) is str:
            if res == "niepsi":
                return pn.pane.HTML("<center><h2>Nie sú tu psi.</h2></center>", width=1000, height=600)
            return pn.pane.HTML("<center><h2>Problém s menom ulice, nie je v DB.</h2></center>", width=1000, height=600)
        return res    # ul_map
    else:
        ul_map = psiMesta(mv, mesta[mv]['psi'], plemeno_vyber, mapa=True)
        return ul_map

In [ ]:
ovladace = pn.Column(zobraz_vyber, ulica_vyber, plemeno_vyber)
zobrazenie = pn.panel(psi_zobraz, width=1000, height=600, align="center")


In [ ]:
nadpis_celkovy = pn.Row(pn.Spacer(width=100),
                        pn.pane.Markdown("##Psi v meste (Prešov, Žilina, Martin)<br><br>", width=500))
appka = pn.Column(nadpis_celkovy, mesto_vyber, pn.Row(pn.Spacer(width=50), ovladace, zobrazenie))
appka.servable(title="Psi v meste")